In [2]:
import pandas as pd
import os

#### Parsing conversations

In [25]:
all_columns=['conversation_id', 'id','author_id','text','in_reply_to_user_id']
for file in os.listdir("conversations"):
    if file.split(".")[1]=="json":
        print("--------------Processing file {0}--------------".format(file))
        df=pd.read_json(os.path.join("conversations",file),lines=True)
        all_tweets=[]
        i=100
        for line in df['data']:
            for tweet in line:
                tweet_info=[]
                for column in all_columns:
                    tweet_info.append(tweet[column])
                try:
                    if tweet['referenced_tweets'][0]['type']=='replied_to':
                        tweet_info.append(tweet['referenced_tweets'][0]['id'])
                    else:
                        tweet_info.append('')
                except:
                    print(tweet['referenced_tweets'][0])
                    tweet_info.append('')
                all_tweets.append(tweet_info)
            if len(all_tweets)>i:
                print("--------------Processed more than {0} tweets--------------".format(i))
                i=i*10
        df_out = pd.DataFrame(all_tweets,columns=all_columns+['replied_to'])
        df_out.to_csv("Parsed_conversations/"+file.split(".")[0]+".csv", index=False)

--------------Processing file Australia_ids.json--------------
--------------Processed more than 100 tweets--------------
--------------Processed more than 1000 tweets--------------
--------------Processing file Britain_ids.json--------------
--------------Processed more than 100 tweets--------------
--------------Processed more than 1000 tweets--------------
--------------Processed more than 10000 tweets--------------
--------------Processing file Canada_ids.json--------------
--------------Processed more than 100 tweets--------------
--------------Processed more than 1000 tweets--------------
--------------Processed more than 10000 tweets--------------
--------------Processing file ICRC_library.json--------------
--------------Processed more than 100 tweets--------------
--------------Processing file IFRC_Europe.json--------------
--------------Processed more than 100 tweets--------------
--------------Processed more than 1000 tweets--------------
--------------Processing file Irelan